In [2]:
import os
import pathlib
import sys

In [5]:
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [6]:
from src import feature_generator, numerical, categorical, dispatcher, cross_validation, utils, visualize

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression

In [8]:
wteams = pd.read_csv(f'{dispatcher.WDSTAGE1_TEAMS}')

In [11]:
wteams.TeamName.nunique()

365

In [13]:
wteams.loc[wteams.TeamID == 3467]

,TeamID,TeamName
364,3467,Merrimack


In [15]:
wseasons = pd.read_csv(f'{dispatcher.WDSTAGE1_SEASONS}')

In [16]:
wseasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West
2,2000,11/1/1999,East,Midwest,Mideast,West
3,2001,10/30/2000,East,Midwest,Mideast,West
4,2002,10/29/2001,East,West,Mideast,Midwest


In [17]:
wseasons.shape

(23, 6)